# Using 🤗 PEFT & bitsandbytes to finetune a LoRa checkpoint




In [1]:
# !pip install -q bitsandbytes datasets accelerate loralib
# !pip install datasets
# !pip install -q git+https://github.com/huggingface/transformers.git@main git+https://github.com/huggingface/peft.git

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [1]:
from huggingface_hub import notebook_login

notebook_login()

In [2]:
!nvidia-smi -L

GPU 0: Tesla T4 (UUID: GPU-d4ab3b72-ddcf-482b-0676-b561db54234d)


### Setup the model

In [4]:
# !pip install datasets # Make sure the datasets module is installed

In [15]:
!pip install evaluate

In [16]:
import transformers
from datasets import load_dataset
from evaluate import load as load_metric  # Corrected import for metrics
from transformers import AutoTokenizer, AutoModelForTokenClassification, TrainingArguments, Trainer, DataCollatorForTokenClassification, EarlyStoppingCallback
import torch


In [14]:
# !pip install transformers==4.44.2

  Using cached transformers-4.44.2-py3-none-any.whl.metadata (43 kB)
  Using cached tokenizers-0.19.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.7 kB)
Using cached transformers-4.44.2-py3-none-any.whl (9.5 MB)
Using cached tokenizers-0.19.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (3.6 MB)
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.13.3
    Uninstalling tokenizers-0.13.3:
      Successfully uninstalled tokenizers-0.13.3
  Attempting uninstall: transformers
    Found existing installation: transformers 4.30.0
    Uninstalling transformers-4.30.0:
      Successfully uninstalled transformers-4.30.0


In [5]:
# import os
# os.environ["CUDA_VISIBLE_DEVICES"]="0"
# os.environ['CUDA_LAUNCH_BLOCKING'] = '1'  # For detailed CUDA error messages

# import torch
# import torch.nn as nn
# import bitsandbytes as bnb
# from transformers import AutoTokenizer, AutoConfig, AutoModelForCausalLM,AutoModelForTokenClassification
# from peft import LoraConfig, get_peft_model
# from transformers.quantization import BitsAndBytesConfig


# # Define the quantization configuration using BitsAndBytes
# quantization_config = BitsAndBytesConfig(
#     device_map='auto',               # Automatically maps layers to available GPUs
#     offload_config={'cpu_offload': True},  # Enables offloading to CPU if GPU memory is insufficient
#     torch_dtype=torch.float16,       # Uses float16 for reduced precision and memory footprint
#     attn_implementation='flash_attention_2'  # Use flash attention implementation for efficiency
# )


# # Load the model with the new quantization configuration
# model = AutoModelForTokenClassification.from_pretrained(
#     "FacebookAI/xlm-roberta-large",
#     num_labels=9,                   # Assuming the number of labels for your NER task
#     quantization_config=quantization_config  # Apply the quantization configuration
# )

# tokenizer = AutoTokenizer.from_pretrained("FacebookAI/xlm-roberta-large")


ModuleNotFoundError: No module named 'transformers.quantization'

In [6]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"  # For detailed CUDA error messages

import torch
from transformers import AutoTokenizer, AutoModelForTokenClassification

# Load the tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("FacebookAI/xlm-roberta-large")
model = AutoModelForTokenClassification.from_pretrained(
    "FacebookAI/xlm-roberta-large",
    num_labels=9  # Number of labels for the NER task
)

# Manually set model to appropriate device and apply quantization if necessary
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

if torch.cuda.is_available():
    model.half()  # Apply half precision (equivalent to float16)

# If you have specific device management or quantization needs, they will need to be handled manually.


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


model.safetensors:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

Some weights of XLMRobertaForTokenClassification were not initialized from the model checkpoint at FacebookAI/xlm-roberta-large and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


### Freezing the original weights


In [9]:
# for param in model.parameters():
#   param.requires_grad = False  # freeze the model - train adapters later
#   if param.ndim == 1:
#     # cast the small parameters (e.g. layernorm) to fp32 for stability
#     param.data = param.data.to(torch.float32)

# model.gradient_checkpointing_enable()  # reduce number of stored activations
# model.enable_input_require_grads()

# class CastOutputToFloat(nn.Sequential):
#   def forward(self, x): return super().forward(x).to(torch.float32)
# model.lm_head = CastOutputToFloat(model.lm_head)

### Setting up the LoRa Adapters

In [7]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

In [8]:
from peft import LoraConfig, get_peft_model

config = LoraConfig(
    r=16, #attention heads
    lora_alpha=32, #alpha scaling
    # target_modules=["q_proj", "v_proj"], #if you know the
    lora_dropout=0.05,
    bias="none",
     task_type="default"  # Assuming 'default' is a neutral option
)

model = get_peft_model(model, config)
print_trainable_parameters(model)

trainable params: 1572864 || all params: 560422921 || trainable%: 0.2806566150423387


## Data

In [9]:
# load data
data = load_dataset("procit008/training_dataset_ner_0.2")
# Get label list
label_list = data["train"].features["ner_tags"].feature.names

README.md:   0%|          | 0.00/2.33k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/4.38M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/716k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/686k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/155178 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/20892 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/21098 [00:00<?, ? examples/s]

In [10]:
data


DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 155178
    })
    validation: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 20892
    })
    test: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 21098
    })
})

In [11]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)
    labels = []
    for i, label in enumerate(examples["ner_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            else:
                label_ids.append(-100)
            previous_word_idx = word_idx
        labels.append(label_ids)
    tokenized_inputs["labels"] = labels
    return tokenized_inputs

tokenized_datasets = data.map(tokenize_and_align_labels, batched=True)

Map:   0%|          | 0/155178 [00:00<?, ? examples/s]

Map:   0%|          | 0/20892 [00:00<?, ? examples/s]

Map:   0%|          | 0/21098 [00:00<?, ? examples/s]

In [15]:
# # Load pre-trained model
# from transformers import BitsAndBytesConfig, AutoConfig,AutoModelForTokenClassification,AutoModelForMaskedLM

# checkpoint = "facebook/xlm-roberta-large"

# config = AutoConfig.from_pretrained("FacebookAI/xlm-roberta-large", num_labels=9)
# quant_config = BitsAndBytesConfig(load_in_8bit=True)

# model = AutoModelForTokenClassification.from_pretrained(
#     checkpoint,
#     config=config,
#     quantization_config=quant_config,
#     torch_dtype=torch.bfloat16,
#     attn_implementation="flash_attention_2",
#     device_map="auto"
# )

In [16]:
# !pip install datasets --upgrade

In [18]:
!pip install seqeval


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 3.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16161 sha256=5e07b543cda64681a7cb65579fbc9e57e809fd7ec065e0f5c4da0b9fc302ed7d
  Stored in directory: /root/.cache/pip/wheels/1a/67/4a/ad4082dd7dfc30f2abfe4d80a2ed5926a506eb8a972b4767fa
Successfully built seqeval


In [19]:
# Load seqeval metric
metric = load_metric("seqeval")

# Define compute_metrics function
def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (special tokens)
    true_labels = [[label_list[l] for l in label if l != -100] for label in labels]
    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

### Training

In [20]:
# Set up data collator for dynamic padding
data_collator = DataCollatorForTokenClassification(tokenizer)

In [21]:
from transformers import AdamW, get_scheduler, TrainingArguments, Trainer, EarlyStoppingCallback

# Assuming model, tokenizer, data_collator, tokenized_datasets, and compute_metrics are already defined

# Set up training arguments
training_args = TrainingArguments(
    output_dir="./FacebookAI/xlm-roberta-large-NER_sep25",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,  # Weight decay for L2 regularization
    load_best_model_at_end=True,  # Load the best model when early stopping is triggered
    metric_for_best_model="f1",  # Metric to monitor for early stopping
    greater_is_better=False,

)

In [22]:
# Initialize an optimizer
optimizer = AdamW(
    model.parameters(),
    lr=training_args.learning_rate,
    weight_decay=training_args.weight_decay
)

# Define the scheduler (optional, here using a linear scheduler with warmup)
lr_scheduler = get_scheduler(
    name="linear",
    optimizer=optimizer,
    num_warmup_steps=0,  # Number of warmup steps (you can change this)
    num_training_steps=len(tokenized_datasets["train"]) // training_args.per_device_train_batch_size * training_args.num_train_epochs,
)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [23]:
# Initialize Trainer with compute_metrics and data collator
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    optimizers=(optimizer, lr_scheduler),  # Pass the optimizer and scheduler to the Trainer
   # callbacks=[early_stopping_callback]  # Include early stopping callback
)

In [ ]:
# Train model
trainer.train()

Epoch,Training Loss,Validation Loss


In [ ]:
trainer.push_to_hub("procit008/FacebookAI/xlm-roberta-large-NER_sep25")
tokenizer.push_to_hub("procit008/FacebookAI/xlm-roberta-large-NER_sep25")

## Share adapters on the 🤗 Hub

In [ ]:
model.push_to_hub("procit008/FacebookAI/xlm-roberta-large-NER_sep25",
                  use_auth_token=True,
                  commit_message="basic training",
                  private=True)

## Load adapters from the Hub

In [ ]:
import torch
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM, AutoTokenizer

peft_model_id = "samwit/bloom-7b1-lora-tagger"
config = PeftConfig.from_pretrained(peft_model_id)
model = AutoModelForCausalLM.from_pretrained(config.base_model_name_or_path, return_dict=True, load_in_8bit=True, device_map='auto')
tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)

# Load the Lora model
model = PeftModel.from_pretrained(model, peft_model_id)

## Inference

In [ ]:
batch = tokenizer("“Training models with PEFT and LoRa is cool” ->: ", return_tensors='pt')

with torch.cuda.amp.autocast():
  output_tokens = model.generate(**batch, max_new_tokens=50)

print('\n\n', tokenizer.decode(output_tokens[0], skip_special_tokens=True))